In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import mean, min, max, rand, randn

In [ ]:
spark = SparkSession \
        .builder \
        .appName('demo') \
        .getOrCreate()

In [ ]:
# alt ways to create a spark dataframe
a = spark.createDataFrame(["10","11","13"], "string").toDF("age")
a.show()
#alt_way_b = spark.createDataFrame([("10", ), ("11", ), ("13",  )], ["age"])
#from pyspark.sql.types import StringType
#spark.createDataFrame(["10","11","13"], StringType()).toDF("age")

In [ ]:
# df = spark.range(0, 10)
# df.select("id", rand(seed=10).alias("uniform"), randn(seed=27).alias("normal")).show()

df = spark \
        .range(0, 10) \
        .withColumn('uniform', rand(seed=13)) \
        .withColumn('normal', randn(seed=31)) \
        .withColumn('normal', randn(seed=31))

In [ ]:
#df.select("id", rand(seed=10).alias("uniform"), randn(seed=27).alias("normal")).show()

In [ ]:
new_row = spark.createDataFrame([(10, 1, 1)], ['id', 'uniform', 'normal'])
df = df.union(new_row)

In [ ]:
df.printSchema()
df.show()

In [ ]:
df.select([mean('uniform'), min('uniform'), max('uniform')]).show()

In [ ]:
# sample covariance of two columns
df.stat.cov('uniform', 'normal')

In [ ]:
# events
e_cols = ["EVENTID","EVENTTIMESTAMP","EVENTTYPE","PROVIDERID","PATIENTID","TASKID","ENCOUNTERID","WORKSTATIONID","PROCESS_ID","EVENTACTION"]
e_rows = [
    ("5636059882.643001","2019-08-07 00:11:22","View","14173","Z854894","17008","30041094735","10003000","prd-32835774","QUERY"),
    ("5636059882.643001","2019-08-07 00:11:22","View","14173","Z854894","17008","30041094735","99001647","prd-32835774","QUERY"),
    ("5636059882.643001","2019-08-07 00:11:22","View","14173","Z854894","17008","30041094735","99001647","prd-32835774","QUERY"),
    ("5636059882.643001","2019-08-07 00:11:22","View","14173","Z854894","17008","30041094735","20000073","prd-32835774","QUERY"),
    ("5636059882.643001","2019-08-07 00:11:22","View","14173","Z854894","17008","30041094735","","prd-32835774","QUERY")
]

# workstations
w_cols = ["WORKSTATIONID","WORKSTATIONAREA","EXTERNAL_NAME","DEPARTMENT_ID","NETWORK","COMPANY"]
w_rows = [
    ("10003000","BDW ORTHO/NEURO","Brandywine Hospital Ortho/Neuro","107001009","Brandywine","Brandywine Hospital"),
    ("20000073","FAMILY FLOURTOWN THMG","Springfield Medical Associates","102409001","Chestnut Hill","CHN - Chestnut Hill Clinic"),
    ("00195B2FF60701080B160C03020E1809","EMH EMERGENCY","EMH Emergency Department","10101100","",""),
    ("40001111","PHX 2AR","Rehabilitation at Phoenixville Hospital","107601028","Phoenixville","Phoenixville Hospital"),
    ("50001297","PTN MAIN OR","Pottstown Main OR","107801016","Pottstown","Pottstown Hospital"),
    ("50001303","PTN EMERGENCY","Pottstown Hospital Emergency Department","107801001","Pottstown","Pottstown Hospital"),
    ("50001310","INTL MED BOYER THMG","Berks Medical Associates","103201001","Pottstown","PTS - Pottstown Clinic Co")
]

e_df = spark.createDataFrame(e_rows, e_cols)
w_df = spark.createDataFrame(w_rows, w_cols)

lte_df = w_df.select(w_df['WORKSTATIONID'], w_df['NETWORK'])

In [ ]:
e_df.printSchema()
e_df.show()

In [ ]:
foo = w_df.filter()select(w_df['WORKSTATIONID'], w_df['NETWORK'])

In [ ]:
newRow = spark.createDataFrame([("4", "ASD")], ['WORKSTATIONID', 'NETWORK'])
appended = lte_df.union(newRow)
appended.show()

# .filter(df.age > 3)

In [ ]:
w_df.printSchema()
w_df.show()

In [ ]:
df = e_df.join(lte_df, "WORKSTATIONID", "right_outer")

In [ ]:
df.filter(df['EVENTID'] != '').show()

In [ ]:
from pyspark.sql.functions import round
a = df.groupBy('NETWORK').count()

def pctVal(x):
    #return round(a['count']/df.count(),2)
    b = round(x/df.count(),2)
    return b
    
a.withColumn('pct', round(a['count']/df.count(), 2)).show()

In [ ]:
def func(v):
    print(a[v])
    return v

a.select('NETWORK', func('count')).show()